In [1]:
import numpy as np
from sklearn.feature_extraction import image
import imageio
from matplotlib.pyplot import imshow
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import os
import random
from tqdm import tqdm

# use this to split a dataset in one folder into training and validation folders
def splitAndMoveData(data_folder, train_folder="data/training/", val_folder="data/validation/", val_split=0.1):
    filenames = os.listdir(data_folder)
    random.shuffle(filenames)

    train_files = filenames[:len(filenames) - int(len(filenames) * val_split)]
    val_files = filenames[len(filenames) - int(len(filenames) * val_split):]

    print("{} training files and {} validation files".format(len(train_files), len(val_files)))

    for f in tqdm(train_files):
        os.replace(data_folder + f, train_folder + f)

    for f in tqdm(val_files):
        os.replace(data_folder + f, val_folder + f)

# use this to get the training and validation file paths for the data generators
def getFilePaths(train_folder="data/training/", val_folder="data/validation/"):
    train_files = [train_folder + f for f in os.listdir(train_folder)]
    val_files = [val_folder + f for f in os.listdir(val_folder)]

    print("{} training files and {} validation files".format(len(train_files), len(val_files)))

    return train_files, val_files



In [30]:
import cv2
import imageio
import numpy as np
from sklearn.feature_extraction import image
import keras
from PIL import Image
import random

class data_generator(keras.utils.Sequence):
    def __init__(self):
        self.patch_size = patch_size
        self.patch_h = patch_size[1]
        self.patch_w = patch_size[0]
        self.x_stride = x_stride
        self.y_stride = y_stride
        

    def __len__(self) :
        return (np.ceil(len(self.filenames) / float(self.batch_size))).astype(np.int)
    def __iter__(self):
        """Create a generator that iterate over the Sequence."""
        while 1:
            for item in (self[i] for i in range(len(self))):
                yield item

    def load(self, path):
        #img = Image.open(path)
        #img = np.array(img, dtype=np.uint8)
        img = keras.preprocessing.image.load_img(path)
        img = keras.preprocessing.image.img_to_array(img)

        return img / 255.
    
    def pixalate_image(self, image, scale_percent = 40):
      og_w = image.shape[1]
      og_h = image.shape[0]
      width = int(image.shape[1] * scale_percent / 100)
      height = int(image.shape[0] * scale_percent / 100)
      dim = (width, height)

      small_image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
      
      # scale back to original size
      width = og_w
      height = og_h
      dim = (width, height)

      low_res_image = cv2.resize(small_image, dim, interpolation = cv2.INTER_AREA)

      return low_res_image
    
    def getPatchRanges(self, img):
      ranges = []
      for y in range(self.patch_h, img.shape[0] + 1 + (self.y_stride - (img.shape[0] % self.y_stride)), self.y_stride):
        if y > img.shape[0]:
          y = img.shape[0]

        for x in range(self.patch_w, img.shape[1] + 1 + (self.x_stride - (img.shape[1] % self.x_stride)), self.x_stride):
          if x > img.shape[1]:
            x = img.shape[1]

          ranges.append([y - self.patch_h, y, x - self.patch_w, x])

      return ranges

    def extractPatches(self, img, ranges):
      return [img[y0 : y1, x0 : x1] for y0, y1, x0, x1 in ranges]

    def __getitem__(self, idx):
        batch_ids = self.filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        og_images = [self.load(f) for f in batch_ids]#np.array(list(map(self.load, batch_ids)))
        pixelated_images = [self.pixalate_image(img, 25) for img in og_images]#np.array(list(map(self.pixalate_image, images)))

        if not self.train:
          np.random.seed(0)

        target_patches = []
        input_patches = []
        for i, img in enumerate(og_images):
          if not self.train:
            ranges = self.getPatchRanges(img)
            idxs = np.random.choice(list(range(len(ranges))), size = max_patches)
            ranges = [ranges[i] for i in idxs]
          else:
            ranges = random.sample(self.getPatchRanges(img), max_patches)

          target_patches.extend(self.extractPatches(img, ranges))
          input_patches.extend(self.extractPatches(pixelated_images[i], ranges))
        
        target_patches = np.array(target_patches)
        input_patches = np.array(input_patches)
        return input_patches, target_patches
        
class train_generator(data_generator):
  def __init__(self):
    self.train = True
    self.filenames = train_files
    self.batch_size = batch_size
    self.max_patches = max_patches
    super().__init__()

class val_generator(data_generator):
  def __init__(self):
    self.train = False
    self.filenames = val_files
    self.batch_size = val_batch_size
    self.max_patches = val_max_batches
    super().__init__()

# loader functions for the generators needed by tensorflow
# in order to use interleave   
def get_train_dataset(self):
    train = True
    self = tf.data.Dataset.from_generator(
        train_generator,
        output_types = (tf.float32, tf.float32))
    return self

def get_val_dataset(self):
    train = False
    self = tf.data.Dataset.from_generator(
        val_generator,
        output_types = (tf.float32, tf.float32))
    return self

In [31]:
#@title
from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D
from tensorflow.keras.callbacks import EarlyStopping

def getModel():
  Input_img = Input(shape = patch_size + (3,))  
      
  #encoding architecture
  x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(Input_img)
  x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x1)
  x2 = MaxPool2D( (2, 2))(x2)
  encoded = Conv2D(64, (3, 3), activation='relu', padding='same')(x2)

  # decoding architecture
  x3 = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
  x3 = UpSampling2D((2, 2))(x3)
  x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x3)
  x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(x2)
  decoded = Conv2D(3, (3, 3), padding='same')(x1)

  autoencoder = Model(Input_img, decoded)
  #autoencoder.summary()
  autoencoder.compile(optimizer='adam', loss='mse')

  return autoencoder

In [32]:
import os
import keras

train_files, val_files = getFilePaths()

patch_size = (100, 100)
y_stride = 25
x_stride = 25
batch_size = 16 # number of images to get patches from
patches_per_batch = 32 # total number of patches per batch of images
max_patches = patches_per_batch // batch_size   # number of patches to get from each image in the batch
train_steps = len(train_files) // batch_size

# seperate batch size for validation so that you can include more patches per image
val_batch_size = 4
val_max_batches = patches_per_batch // val_batch_size
val_steps =  len(val_files) // val_batch_size

32850 training files and 3650 validation files


In [33]:
#@title
# from each of the generators create a pair of interleaved datasets
# tensorflow can automatically multiprocess interleaved datasets
# so that while batches can be loaded and processed ahead of time
interleaved_train = tf.data.Dataset.range(2).interleave(
                    get_train_dataset,
                    num_parallel_calls=tf.data.experimental.AUTOTUNE
                )
interleaved_val = tf.data.Dataset.range(2).interleave(
                    get_val_dataset,
                    num_parallel_calls=tf.data.experimental.AUTOTUNE
                )

In [49]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
patience=3,
mode='min', 
restore_best_weights=True),
tf.keras.callbacks.ModelCheckpoint("data/model_checkpoints/weights.{epoch:02d}-{val_loss:.2f}.hdf5",
monitor='val_loss', 
save_best_only=True,
mode='min',
save_weights_only=True)
]

autoencoder = getModel()

# set this to none if starting from scratch
model_weights = None

if model_weights:
    autoencoder.load_weights(model_weights)

a_e = autoencoder.fit(interleaved_train,
                   steps_per_epoch = train_steps,
                   epochs = 10,
                    verbose=1,
                    validation_data=interleaved_val,
                    validation_batch_size=val_batch_size,
                    validation_steps=val_steps,
                   callbacks=callbacks)

autoencoder.save("data/saved_models/encoder1.h5")

Epoch 1/10
2053/2053 [==============================] - 1074s 523ms/step - loss: 0.0043 - val_loss: 0.0032
Epoch 2/10
2053/2053 [==============================] - 1068s 520ms/step - loss: 0.0047 - val_loss: 0.0044
Epoch 3/10
2053/2053 [==============================] - 1071s 522ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 4/10
2053/2053 [==============================] - 1033s 503ms/step - loss: 0.0033 - val_loss: 0.0033


In [57]:
from tqdm import tqdm

def reconstruct_image(patches, ranges, weights):
  img = np.zeros(weights.shape)
  for i, (y0, y1, x0, x1) in enumerate(ranges):
    img[y0:y1,x0:x1] += patches[i]
  
  return np.array(np.clip((img / weights) * 255, 0.0, 255.0), dtype=np.uint8)

def sharpen_image(model, img):
  patches = []
  weights = np.zeros(img.shape)
  ranges = []
  for y in range(patch_size[1], img.shape[0] + 1 + (y_stride - (img.shape[0] % y_stride)), y_stride):
    if y > img.shape[0]:
      y = img.shape[0]

    for x in range(patch_size[0], img.shape[1] + 1 + (x_stride - (img.shape[1] % x_stride)), x_stride):
      if x > img.shape[1]:
        x = img.shape[1]

      y0 = y - patch_size[1]
      x0 = x - patch_size[0]
      patches.append(img[y0 : y, x0 : x])
      weights[y0 : y, x0 : x] += 1
      ranges.append([y0, y, x0, x])

  patches = np.array(patches)
  patch_batch_count = batch_size * max_patches
  preds = []
  for i in tqdm(list(range(0, patches.shape[0], patch_batch_count))):
    preds.append(np.array(model.predict(patches[i : i + patch_batch_count])))
  
  preds = np.concatenate(preds)

  return reconstruct_image(preds, ranges, weights)

def pixalate_image(image, scale_percent = 40):
      width = int(image.shape[1] * scale_percent / 100)
      height = int(image.shape[0] * scale_percent / 100)
      dim = (width, height)

      small_image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
      
      # scale back to original size
      width = image.shape[1]
      height = image.shape[0]
      dim = (width, height)

      low_res_image = cv2.resize(small_image, dim, interpolation = cv2.INTER_AREA)

      return low_res_image
  


In [59]:
from timeit import default_timer as timer
filenames = os.listdir('data/tests/')

start = timer()

# put some images in the data/tests/ path and this will pixelate the model will upscale them
for i, f in enumerate(filenames):
    img = keras.preprocessing.image.load_img("data/tests/" + f)
    img = keras.preprocessing.image.img_to_array(img) / 255.
    img = pixalate_image(img, 25)
    img = np.array(img)
    #imageio.imwrite("data/test_outputs/{}lr.png".format(i + 1), np.array(img * 255., dtype=np.uint8))
    sharp_img = sharpen_image(autoencoder, img)
    imageio.imsave("data/test_outputs/{}.png".format(i + 1), np.hstack([np.array(img * 255., dtype=np.uint8), sharp_img]))

print((timer() - start) / len(filenames))

100%|██████████| 15/15 [00:02<00:00,  5.81it/s]
3.793466590909091
